#### Question: Generating Fashion Products with GAN
Dataset Problem: Use the Fashion MNIST dataset to train a GAN for generating new fashion product images. 
The Fashion-MNIST dataset is proposed as a more challenging replacement dataset for the
MNIST dataset. It is a dataset comprised of 60,000 small square 28 Å~ 28-pixel grayscale images
of items of 10 types of clothing, such as shoes, t-shirts, dresses, and more. The mapping of all
0-9 integers to class labels are listed below.

0: T-shirt/top

1: Trouser

2: Pullover

3: Dress

4: Coat

5: Sandal

6: Shirt

7: Sneaker

8: Bag

9: Ankle boot

Download data from: https://github.com/zalandoresearch/fashion-mnist



In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  
])
transform

Compose(
    ToTensor()
    Normalize(mean=(0.5,), std=(0.5,))
)

In [5]:
dataset = datasets.FashionMNIST(root='data', train=True, transform=transform, download=True)
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)
dataloader

In [6]:
class Generator(nn.Module):
    def __init__(self, noise_dim=100, img_dim=28*28):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(noise_dim, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, img_dim),
            nn.Tanh()
        )

    def forward(self, x):
        return self.model(x)

In [7]:
class Discriminator(nn.Module):
    def __init__(self, img_dim=28*28):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(img_dim, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [9]:
noise_dim = 100
generator = Generator(noise_dim).to(device)
discriminator = Discriminator().to(device)

In [10]:
criterion = nn.BCELoss()
lr = 0.0002
optimizer_G = optim.Adam(generator.parameters(), lr=lr)
optimizer_D = optim.Adam(discriminator.parameters(), lr=lr)

In [11]:
epochs = 50
for epoch in range(epochs):
    for batch_idx, (real, _) in enumerate(dataloader):
        real = real.view(-1, 28*28).to(device)
        batch_size = real.size(0)

        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

     
        noise = torch.randn(batch_size, noise_dim).to(device)
        fake = generator(noise)
        output_real = discriminator(real)
        output_fake = discriminator(fake.detach())

        loss_D_real = criterion(output_real, real_labels)
        loss_D_fake = criterion(output_fake, fake_labels)
        loss_D = loss_D_real + loss_D_fake

        optimizer_D.zero_grad()
        loss_D.backward()
        optimizer_D.step()

      
        output = discriminator(fake)
        loss_G = criterion(output, real_labels)  # Generator wants D(fake)=1
        optimizer_G.zero_grad()
        loss_G.backward()
        optimizer_G.step()

    print(f"Epoch [{epoch+1}/{epochs}]  Loss D: {loss_D:.4f}, Loss G: {loss_G:.4f}")

Epoch [1/50]  Loss D: 0.2296, Loss G: 2.8179
Epoch [2/50]  Loss D: 0.3620, Loss G: 5.6721
Epoch [3/50]  Loss D: 0.4158, Loss G: 4.4276
Epoch [4/50]  Loss D: 0.2564, Loss G: 3.5288
Epoch [5/50]  Loss D: 0.1926, Loss G: 4.8739
Epoch [6/50]  Loss D: 0.1746, Loss G: 3.5088
Epoch [7/50]  Loss D: 0.4021, Loss G: 4.5513
Epoch [8/50]  Loss D: 0.3746, Loss G: 2.9444
Epoch [9/50]  Loss D: 0.4482, Loss G: 4.7099
Epoch [10/50]  Loss D: 0.5077, Loss G: 3.2428
Epoch [11/50]  Loss D: 0.6570, Loss G: 2.8664
Epoch [12/50]  Loss D: 0.5865, Loss G: 2.5005
Epoch [13/50]  Loss D: 0.4031, Loss G: 3.0745
Epoch [14/50]  Loss D: 0.5773, Loss G: 2.9036
Epoch [15/50]  Loss D: 0.7744, Loss G: 2.2737
Epoch [16/50]  Loss D: 0.4576, Loss G: 2.6656
Epoch [17/50]  Loss D: 0.5132, Loss G: 3.2202
Epoch [18/50]  Loss D: 0.6003, Loss G: 2.7080
Epoch [19/50]  Loss D: 0.9478, Loss G: 2.0170
Epoch [20/50]  Loss D: 0.5742, Loss G: 1.8613
Epoch [21/50]  Loss D: 0.7986, Loss G: 2.0986
Epoch [22/50]  Loss D: 0.5671, Loss G: 2.44

In [ ]:
with torch.no_grad():
    noise = torch.randn(25, noise_dim).to(device)
    fake_images = generator(noise).reshape(-1, 1, 28, 28)
    fake_images = (fake_images + 1) / 2.0  # rescale to [0, 1]

    plt.figure(figsize=(5, 5))
    for i in range(25):
        plt.subplot(5, 5, i+1)
        plt.imshow(fake_images[i][0].cpu(), cmap='gray')
        plt.axis('off')
    plt.show()